# Clean Player Data
The player_data.csv dataset is an export from Basketball Monster. This cleansing process prepares the player_data dataset for insertion into a PostgreSQL instance hosted on AWS RDS.

In [327]:
import pandas as pd
import numpy as np

# Setup prettier printing
pd.set_option('expand_frame_repr', False)

In [328]:
# Read in the player dataset
player_df = pd.read_csv(
	'../src/data/raw/player_data.csv',
		sep=',',
		header=0,
		encoding='utf-8',
		skip_blank_lines=True,
		dtype={
			'Name': pd.StringDtype(),
			'Team': pd.StringDtype(),
			'Pos': pd.StringDtype(),
			'Draft': pd.StringDtype()
		}
)
print(player_df.sample(5))

               Name Team   Pos   Age    Draft
384  Tremont Waters  BOS    PG  23.5  2019 51
199  Xavier Tillman  MEM  PF-C  22.5  2020 35
420      Trey Lyles  SAS  PF-C  25.7  2015 12
295     Aron Baynes  TOR     C  34.6  2009 20
328   Frank Jackson  DET    PG  23.2  2017 31


## Preprocessing
Here we conduct some standard preparation for the dataset:
* Converting all column headings to lowercase for consistent column access

In [329]:
# Convert headings to lowercase
player_df.columns = player_df.columns.str.lower()
print(player_df.columns)


Index(['name', 'team', 'pos', 'age', 'draft'], dtype='object')


## Data Cleaning
* Check for null data

In [330]:
# Check for null values
player_df.isnull().sum()


name     0
team     0
pos      0
age      0
draft    0
dtype: int64

## Data Transformation
* Check for duplicate data
* Check for irregular data
	* Team abbreviations
	* Player ages

In [331]:
# Check for duplicate players
player_df.duplicated(subset=['name']).sum()

0

In [332]:
# Check for irregular team abbreviations
player_df['team'].sort_values().unique()

<StringArray>
['ATL', 'BKN', 'BOS', 'CHA', 'CHI', 'CLE', 'DAL', 'DEN', 'DET',  'FA', 'GSW',
 'HOU', 'IND', 'LAC', 'LAL', 'MEM', 'MIA', 'MIL', 'MIN', 'NOR', 'NYK', 'OKC',
 'ORL', 'PHI', 'PHX', 'POR', 'SAC', 'SAS', 'TOR', 'UTA', 'WAS']
Length: 31, dtype: string

We immediately notice that there is a 'FA' designation for team which represents a player that was a free agent at the time the data was gathered. We also notice that New Orleans ('NOR') is misaligned with the official abbreviation ('NOP')

In [333]:
# Transform irregular team abbreviations
player_df['team'].replace({'NOR': 'NOP'}, inplace=True)

In [334]:
# Check for irregular player ages
player_df['age'].sort_values().describe()

count    432.000000
mean      26.839815
std        4.142641
min       19.500000
25%       23.475000
50%       26.300000
75%       29.725000
max       37.400000
Name: age, dtype: float64

Next, we create individual columns from aggregated columns to split name column into first and last names, split the draft column draft year and draft position, and split position into unique position abbreviations (i.e., PG, SG, SF, PF, C).

In [335]:
# Split 'name' column, 'draft' column, and 'pos' columns into respective columns

player_df[['first_name', 'last_name']] = player_df['name'].str.split(' ', 1, expand=True)
player_df[['draft_year', 'draft_pick']] = player_df['draft'].str.split(expand=True)
positions = player_df['pos'].str.split('-', expand=True).rename(
	columns={
		0: 'position_primary',
		1: 'position_secondary',
		2: 'position_tertiary'
	}).fillna(np.nan)

player_df['position_primary'] = positions['position_primary']
player_df['position_secondary'] = positions['position_secondary']
player_df['position_tertiary'] = positions['position_tertiary']

# Use 'age' column to calculate new column 'birth_year'
player_df['age'] = player_df['age'].apply(lambda x: round(2021.55 - x))
player_df.rename(
	columns={
		'age': 'birth_year'
	}, inplace=True)

# Cast draft_year and draft_pick into numeric
player_df['draft_year'] = pd.to_numeric(player_df['draft_year'], downcast='integer')
player_df['draft_pick'] = pd.to_numeric(player_df['draft_pick'], downcast='integer')

# Check the datatypes
print(player_df.dtypes)


name                   string
team                   object
pos                    string
birth_year              int64
draft                  string
first_name             string
last_name              string
draft_year              int16
draft_pick            float64
position_primary       string
position_secondary     string
position_tertiary      string
dtype: object


In [336]:
# Fill NaN draft picks with 0 to avoid database insertion errors
player_df['draft_pick'].fillna(0, inplace=True)

# Verify no NaN
player_df['draft_pick'].isnull().sum()


0

In [337]:
# Check for irregular draft years and picks
player_df['draft_year'].describe()


count     432.000000
mean     2015.145833
std         4.054104
min      2003.000000
25%      2013.000000
50%      2016.000000
75%      2019.000000
max      2020.000000
Name: draft_year, dtype: float64

In [338]:
player_df.draft_year.unique()

array([2014, 2012, 2009, 2005, 2018, 2013, 2011, 2010, 2017, 2015, 2016,
       2008, 2007, 2019, 2003, 2020, 2006, 2004], dtype=int16)

In [339]:
player_df['draft_pick'].describe()

count    432.000000
mean      21.293981
std       14.114847
min        0.000000
25%       10.000000
50%       20.000000
75%       29.000000
max       60.000000
Name: draft_pick, dtype: float64

In [340]:
player_df.draft_pick.sort_values().unique()

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25.,
       26., 27., 28., 29., 30., 31., 32., 33., 34., 35., 36., 37., 38.,
       39., 40., 41., 42., 43., 44., 45., 46., 47., 48., 49., 50., 51.,
       52., 54., 55., 57., 58., 59., 60.])

We notice there are some gaps in draft picks. We don't have any results for draft picks:
* 53
* 56

In [341]:
# Check for irregular birthyears
player_df.birth_year.describe()

count     432.000000
mean     1994.717593
std         4.144027
min      1984.000000
25%      1992.000000
50%      1995.000000
75%      1998.000000
max      2002.000000
Name: birth_year, dtype: float64

In [342]:
player_df.birth_year.sort_values().unique()

array([1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994,
       1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002])

In [343]:
# Fix faulty birth years for players born in Jan/Dec
modified_birth_years = [
	['Buddy', 'Hield', 1992],
	['LeBron', 'James', 1984],
	['De\'Aaron', 'Fox', 1997],
	['Torrey', 'Craig', 1990],
	['Edmond', 'Sumner', 1995],
	['Gorgui', 'Dieng', 1990],
	['Aleksej', 'Pokusevski', 2001],
	['Eric', 'Gordon', 1988],
	['Sekou', 'Doumbouya', 2000],
	['Jahlil', 'Okafor', 1995],
	['Wesley', 'Iwundu', 1994],
	['Paul', 'Watson', 1994]
]
for i in modified_birth_years:
	# Locate the rows for each of the players in modified_birth_years and update each birth year
	player_df.loc[(player_df['first_name'] == i[0]) & (player_df['last_name'] == i[1]), 'birth_year'] = i[2]

In [358]:
player_df.sort_index(inplace=True)
player_df['player_id'] = player_df.index
player_df.player_id = player_df.player_id + 1
player_df.head(10)

,birth_year,first_name,last_name,draft_year,draft_pick,player_id
0,1996,Nikola,Jokic,2014,41.0,1
1,1991,Damian,Lillard,2012,6.0,2
2,1988,Stephen,Curry,2009,7.0,3
3,1985,Chris,Paul,2005,4.0,4
4,1999,Trae,Young,2018,5.0,5
5,1992,Khris,Middleton,2012,39.0,6
6,1994,Joel,Embiid,2014,3.0,7
7,1995,Giannis,Antetokounmpo,2013,15.0,8
8,1992,Kawhi,Leonard,2011,15.0,9
9,1993,Rudy,Gobert,2013,27.0,10


In [345]:
# Export intermediate data
player_df.to_csv(path_or_buf='../src/data/intermediate/players_intermediate.csv')

In [346]:
# Inspect the current index for the dataset
player_df.index

RangeIndex(start=0, stop=432, step=1)

# Generating Derivative Datasets
In the following steps we use the cleaning steps above to generate additional datasets for:
* **Player position** information which includes:
	* Player index (pandas-generated)
	* Primary position
	* Secondary position
	* Tertiary position
* **Player team** information which includes:
	* Player index (pandas-generated)
	* Team ID
* **Player master** information which includes:
	* Player index (pandas-generated)
	* First name
	* Last name
	* Birth year
	* Draft year
	* Draft pick

## Player Position

In [347]:
# Create a copy of player_data_df for the new dataset
player_position_df = player_df.copy()

In [348]:
# Filter by 2020 season only because positions are only avaiable for the last season
player_position_df['season'] = 2020

# Drop all columns except for the specified columns
player_position_df.drop(player_position_df.columns.difference(['player_id', 'position_primary', 'position_secondary', 'position_tertiary']), axis=1, inplace=True)

# Reorder to match schema
player_position_df = player_position_df[[
	'player_id',
	'position_primary',
	'position_secondary',
	'position_tertiary'
]]

print(player_position_df.head(10))

   player_id position_primary position_secondary position_tertiary
0          0               PF                  C              <NA>
1          1               PG               <NA>              <NA>
2          2               PG                 SG              <NA>
3          3               PG               <NA>              <NA>
4          4               PG               <NA>              <NA>
5          5               SF                 PF              <NA>
6          6               PF                  C              <NA>
7          7               PF                  C              <NA>
8          8               SG                 SF              <NA>
9          9                C               <NA>              <NA>


## Player Team

In [349]:
# Create a copy of player_data_df for the new dataset
player_team_df = player_df.copy()
player_team_df.sample(5)

,name,team,pos,birth_year,draft,first_name,last_name,draft_year,draft_pick,position_primary,position_secondary,position_tertiary,player_id
6,Joel Embiid,PHI,PF-C,1994,2014 3,Joel,Embiid,2014,3.0,PF,C,<NA>,6
239,Dean Wade,CLE,PG-PF-C,1997,2019 20,Dean,Wade,2019,20.0,PG,PF,C,239
386,Chandler Hutchison,WAS,SF-PF,1996,2018 22,Chandler,Hutchison,2018,22.0,SF,PF,<NA>,386
115,Anthony Davis,LAL,PF-C,1993,2012 1,Anthony,Davis,2012,1.0,PF,C,<NA>,115
9,Rudy Gobert,UTA,C,1993,2013 27,Rudy,Gobert,2013,27.0,C,<NA>,<NA>,9


In [350]:
# Read in the dataset
team_df = pd.read_csv(
	'../src/data/intermediate/teams_intermediate.csv',
		sep=',',
		header=0,
		encoding='utf-8',
		skip_blank_lines=True
)
team_df.sample(5)

,team_id,name,shortname,city,state,conference,division
14,1610612764,Wizards,WAS,Washington D.C.,DC,East,Southeast
10,1610612737,Hawks,ATL,Atlanta,GA,East,Southeast
11,1610612766,Hornets,CHA,Charlotte,NC,East,Southeast
2,1610612752,Knicks,NYK,New York City,NY,East,Atlantic
16,1610612750,Timberwolves,MIN,Minneapolis,MN,West,Northwest


In [351]:
# Merge with team_join to get the original id for the team
player_team_df = pd.merge(
	player_team_df,
	team_df,
	left_on='team',
	right_on='shortname',
	how='outer',
	suffixes=['_left', '_right']
)
player_team_df.head(5)

,name_left,team,pos,birth_year,draft,first_name,last_name,draft_year,draft_pick,position_primary,position_secondary,position_tertiary,player_id,team_id,name_right,shortname,city,state,conference,division
0,Nikola Jokic,DEN,PF-C,1996,2014 41,Nikola,Jokic,2014,41.0,PF,C,<NA>,0,1.610613e+09,Nuggets,DEN,Denver,CO,West,Northwest
1,Michael Porter Jr.,DEN,SF-PF,1999,2019 14,Michael,Porter Jr.,2019,14.0,SF,PF,<NA>,31,1.610613e+09,Nuggets,DEN,Denver,CO,West,Northwest
2,Jamal Murray,DEN,PG-SG,1997,2016 7,Jamal,Murray,2016,7.0,PG,SG,<NA>,87,1.610613e+09,Nuggets,DEN,Denver,CO,West,Northwest
3,Facundo Campazzo,DEN,PG,1991,2020,Facundo,Campazzo,2020,0.0,PG,<NA>,<NA>,119,1.610613e+09,Nuggets,DEN,Denver,CO,West,Northwest
4,Will Barton,DEN,SG-SF,1991,2012 40,Will,Barton,2012,40.0,SG,SF,<NA>,147,1.610613e+09,Nuggets,DEN,Denver,CO,West,Northwest


In [352]:
# Drop all columns except for the specified columns
player_team_df.drop(player_team_df.columns.difference(['player_id', 'team_id']), axis=1, inplace=True)


In [353]:
# Check team ids
player_team_df.team_id.unique()

# Replace null team ids with 0
player_team_df.team_id.fillna(0, inplace=True)

# Cast team_id to an int
player_team_df.team_id = player_team_df.team_id.astype(int)

player_team_df.head(5)


,player_id,team_id
0,0,1610612743
1,31,1610612743
2,87,1610612743
3,119,1610612743
4,147,1610612743


## Player Master

In [356]:
# Drop all columns except for the specified columns
player_df.drop(
	player_df.columns.difference(
		['player_id', 'first_name', 'last_name', 'birth_year', 'draft_year', 'draft_pick']
	),
	axis=1,
	inplace=True)
	
player_df.sample(5)

,birth_year,first_name,last_name,draft_year,draft_pick,player_id
205,1998,Payton,Pritchard,2020,26.0,205
162,1997,Terance,Mann,2019,48.0,162
228,1992,Victor,Oladipo,2013,2.0,228
221,1998,Furkan,Korkmaz,2016,26.0,221
429,1992,Rodney,McGruder,2013,20.0,429


In [357]:
player_df.player_id.unique()

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18